# Non-negative Matrix Factorization
## Init

In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from functools import partial

import plotly.express as px


import implicit # Matrix Factorization

from sklearn.decomposition import NMF
from sklearn.model_selection import KFold
from cmfrec import CMF_implicit

In [2]:
import sys
sys.path.append("../")
from src.data_preprocessing import TrainTestGenerator
from src.evaluator import Evaluator

In [3]:
show_ploty = False

In [4]:
data_dir = "../data/"
data_generator = TrainTestGenerator(data_dir)

## Model

In [5]:
# Model wrapper

class CMF_recommender:
    def __init__(self, k=50):
        self.model = CMF_implicit(
            # verbose=False,
            # method="als",
            k=k,
            # lambda_=1e+1
        )

    def fit(self, data: pd.DataFrame):
        data = data.copy()
        data = data.rename(columns={
            "userID": "UserId",
            "artistID": "ItemId",
            "weight": "Rating"
        })
        self.model.fit(data)

    def recommend(self, user_id, n):
        recommendations = self.model.topN(user_id, n=n)
        return recommendations

In [6]:
# Evaluator (forward chaining)

evaluator = Evaluator(CMF_recommender, data_generator)
evaluator.evaluate()

In [7]:
# Get results (hit-rate)

results = evaluator.get_results()
results

,cases,5,10,25,50,500
2008,4556,0.008780,0.018437,0.031826,0.060140,0.220369
2009,4687,0.019842,0.026029,0.046512,0.065714,0.221037
2010,6133,0.008479,0.021197,0.046796,0.082668,0.196152
2011,1129,0.021258,0.046944,0.088574,0.130204,0.313552


## Different numbers of latent features (k)

In [8]:
# Try different numbers of latent features (k): 

results = []
for k in [5, 10, 20, 25, 50, 100]:
    model = partial(CMF_recommender, k=k)
    evaluator = Evaluator(model, data_generator)
    evaluator.evaluate()
    results_partial = evaluator.get_results()
    results_partial["k"] = k
    
    results.append(results_partial)

results = pd.concat(results)

In [9]:
# Hit-rates
results

,cases,5,10,25,50,500,k
2008,4556,0.008560,0.014925,0.037752,0.064969,0.214881,5
2009,4687,0.016855,0.028163,0.057606,0.080862,0.248987,5
2010,6133,0.012718,0.031795,0.052503,0.076308,0.216044,5
2011,1129,0.027458,0.046944,0.097431,0.143490,0.424269,5
2008,4556,0.017120,0.034021,0.053775,0.073529,0.218832,10
2009,4687,0.011308,0.022189,0.041178,0.063793,0.223384,10
2010,6133,0.014838,0.027719,0.045492,0.078917,0.221099,10
2011,1129,0.023915,0.050487,0.091231,0.128432,0.369353,10
2008,4556,0.009877,0.018657,0.035338,0.049824,0.208297,20
2009,4687,0.015575,0.022616,0.038191,0.057179,0.200555,20


In [10]:
results.groupby("k").mean()

,cases,5,10,25,50,500
k,,,,,,
5,4126.25,0.016398,0.030457,0.061323,0.091407,0.276045
10,4126.25,0.016795,0.033604,0.057919,0.086168,0.258167
20,4126.25,0.018422,0.030952,0.051482,0.075167,0.239399
25,4126.25,0.018522,0.031148,0.055969,0.081350,0.234905
50,4126.25,0.014590,0.028152,0.053427,0.084681,0.237777
100,4126.25,0.014693,0.024365,0.048997,0.074828,0.227677


In [11]:
results_data = results.groupby("k").mean().drop(columns="cases").reset_index().melt(id_vars="k", var_name="n", value_name="hit_rate")

if show_ploty:
    fig = px.line(
        results_data,
        x="k", 
        y="hit_rate", 
        color="n"
    )
    fig.show()

In [12]:
if show_ploty:
    px.imshow(results.groupby("k").mean().drop(columns="cases").values)

In [13]:
results_data = results.reset_index().rename(columns={"index": "year"}).melt(id_vars=["year", "cases", "k"], var_name="n", value_name="hit_rate")

In [14]:
if show_ploty:
    fig = px.line(
        results_data[results_data["k"] == 5],
        x="year", 
        y="hit_rate", 
        color="n"
    )
    fig.show()

In [15]:
if show_ploty:
    fig = px.line(
        results_data[results_data["k"] == 25],
        x="year", 
        y="hit_rate", 
        color="n"
    )
    fig.show()

In [16]:
if show_ploty:
    fig = px.line(
        results_data[results_data["k"] == 50],
        x="year", 
        y="hit_rate", 
        color="n"
    )
    fig.show()